Logistic Regression Cow Classifier

In [33]:
#Image classifer with logistic regression
#images shape(width,height,3(rgb))

import numpy as np
import scipy 
import os
import matplotlib.pyplot as plot
import math
from scipy import ndimage
import sys 
sys.path.append("./basicFunctions.py")
# import crossEntropyLoss from basicFunctions

# TO-DO
# fetch a ton more images, of your own choice / category 
# create a train and a test set <---- write code for this 

In [34]:
# Testing cross entropy function
test_x = np.asarray([[1,1,1]])
test_y = np.asarray([[.8,.9,.4]])
crossEntropyLoss(test_x,test_y)


(1, 3) a


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


inf

In [35]:
# Global Constants
# side_length = 650
# image_path = "./untitled folder/"
# test_ratio = .2

In [36]:
side_length = 650
image_path = "./Logistic_Regression_Data/"
test_ratio = .2

In [37]:
def getImageArrays(path, side_length):
    image_names = os.listdir(path)
    examples = []
    for image_name in image_names:
        if image_name.split(".")[-1] != "DS_Store":
            cur_image_path = path + image_name
            cur_array = scipy.ndimage.imread(cur_image_path,flatten=False) #reads image as numpy array (lenght,height,3)
            cur_array_resized = scipy.misc.imresize(cur_array,size=(side_length,side_length,3)) #resizes images to a uniform shape
            examples += [cur_array_resized] 
    return examples
        

In [38]:
#create arrays from images with labels
cow_images_path = image_path + "cows/"
notCow_image_path = image_path + "notcows/"

examples_cow = getImageArrays(cow_images_path, side_length)
labels_cow = np.ones(len(examples_cow))
examples_notCow = getImageArrays(cow_images_path, side_length)
labels_notCow = np.zeros(len(examples_notCow))

examples_cow = np.array(examples_cow)
examples_notCow = np.array(examples_notCow)

assert(examples_cow.shape[1:] == (side_length,side_length,3)), "examples_cow are invalid shape"
print("Examples:",np.shape(examples_cow))
print("Labels:",np.shape(labels_cow))
examples = np.concatenate((examples_cow,examples_notCow))
labels = np.concatenate((labels_cow,labels_notCow))

Examples: (200, 650, 650, 3)
Labels: (200,)


In [39]:
#create train and test examples
print(len(examples))
print(len(labels))
number_test_examples = int(len(examples)*test_ratio)
number_test_labels = int(len(labels)*test_ratio)
print(number_test_examples)
print(number_test_examples)
test_examples = examples[:number_test_examples]
train_examples = examples[number_test_examples:]

labels_test = labels[:number_test_labels]
labels_train = labels[number_test_labels:]

print("test_examples", np.shape(test_examples))
print("test_labels", np.shape(labels_test))
print("train_examples", np.shape(train_examples))
print("train_labels", np.shape(labels_train))

400
400
80
80
test_examples (80, 650, 650, 3)
test_labels (80,)
train_examples (320, 650, 650, 3)
train_labels (320,)


In [40]:
#some final operations of examples
flattened_train_examples = train_examples.reshape(train_examples.shape[0], -1).T
flattened_test_examples = test_examples.reshape(test_examples.shape[0], -1).T

print(np.shape(flattened_train_examples))
print(np.shape(flattened_test_examples))

# Standardize color values of the image (decrease computational cost durring cross entropy)                                                                                                                                                                                                                                                                            
standardized_train_examples = flattened_train_examples/255 #225 is the maximum rgb value/ This is done to decrease varaince in inputs thus more efficint
standardized_test_examples = flattened_test_examples/255
print(standardized_train_examples)

(1267500, 320)
(1267500, 80)
[[ 0.83137255  0.43529412  0.84705882 ...,  0.70196078  0.77254902
   0.53333333]
 [ 0.88627451  0.41568627  0.87058824 ...,  0.65098039  0.80784314
   0.58039216]
 [ 0.92156863  0.4         0.87058824 ...,  0.57647059  0.86666667
   0.32941176]
 ..., 
 [ 0.29019608  0.49019608  0.45882353 ...,  0.38823529  0.38039216
   0.34509804]
 [ 0.40392157  0.47843137  0.45882353 ...,  0.41960784  0.42745098
   0.34901961]
 [ 0.18431373  0.34901961  0.10588235 ...,  0.11764706  0.14509804
   0.12156863]]


In [41]:
# #
# test0 = "./untitled folder/" + "cow0.jpg" #standardize file extensions
# test1 = "./untitled folder/" + "notCow0.jpg" 
# test0Array = scipy.ndimage.imread(test0,flatten=False)
# test1Array = scipy.ndimage.imread(test1,flatten=False) 
# print("Shape of Test Image 0:", np.shape(test0Array))
# print("Shape of Test Image 1:", np.shape(test1Array))

# # Resize input image arrays 
# test0Array_resized = scipy.misc.imresize(test0Array,size=(side_length,side_length,3))
# test1Array_resized = scipy.misc.imresize(test1Array,size=(side_length,side_length,3))

In [42]:
# # Read in input image files into arrays, validate inputs
# images = os.listdir(image_path)

# # First, build dataset for training data
# examples = []
# labels = []
# for image in images:
#     if image.split(".")[-1] != "DS_Store":
#         cur_label = 0
#         if image[0] == "c":
#             cur_label = 1
#         labels += [cur_label]
#         cur_image_path = image_path + image
#         cur_array = scipy.ndimage.imread(cur_image_path,flatten=False) #reads image as numpy array (lenght,height,3)
#         cur_array_resized = scipy.misc.imresize(cur_array,size=(side_length,side_length,3)) #resizes images to a uniform shape
#         examples += [cur_array_resized] 
# # print(labels)

# print("Shape of train_examples:",np.shape(examples)) #should be (len(images),side_length,side_length,3)

# # Then, build dataset for train and test data...
# number_test_examples = int(len(examples)*test_ratio)
# test_examples = examples[:number_test_examples]
# train_examples = examples[number_test_examples:]

# train_examples = np.array(train_examples) #converts the list of examples to an array
# test_examples = np.array(test_examples) 


# # Next, we flatten our train and test examples
# # .T => transposes our inputs
# # (a, b, c, d) ----> (b * c * d, a)

# flattened_train_examples = train_examples.reshape(train_examples.shape[0], -1).T
# flattened_test_examples = test_examples.reshape(test_examples.shape[0], -1).T
# # print(type(train_examples[0]))
# # flattened_train_examples = train_examples.reshape(train_examples[0],-1).T
# #fattened_train_examples = train_examples.flatten()
# print(np.shape(flattened_train_examples))
# print(np.shape(flattened_test_examples))

# # print("Shape of test0Array_resized:",np.shape(test0Array_resized))

# # Standardize color values of the image 
# standardized_train_examples = flattened_train_examples/255 #225 is the maximum rgb value/ This is done to decrease varaince in inputs thus more efficint
# standardized_test_examples = flattened_test_examples/255
# print(standardized_train_examples)


# # Create labels for data
# # labels = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]
# labels_train  = np.array(labels[:21])
# labels_test = np.array(labels[21:])
# # Randomize and shuffle labels/data

In [55]:
def crossEntropyLoss(a,Y):
    print(Y.shape)
    m = Y.shape[1] #len(Y) doesn't work, need 2nd dimesnion
#     a = a.T
    loss = -(1 / m) * np.sum(Y * np.log(a) + (1 - Y) * np.log(1 - a))
    print(loss,"loss")
    return loss

In [44]:
def sigmoid(x):
    output = 1.0/(1.0+np.exp(-x))
    return output
    
# print(sigmoid(np.array([0,2])))
# write test cases for this function so that it works for np arrays 

In [45]:
def initialize_zeros(dimension): #dimension = len(flattened_examples)
    zeroed_weights = np.zeros((dimension, 1)) 
    zeroed_baisies = 0. #baisies always start at 0 b/c they are developed as the model trains
    
    # asserts here, use isinstance()
    # check shape of zeroed weight
    assert(isinstance(zeroed_baisies, float) or isinstance(zeroed_baisies, int))
    
    return zeroed_weights, zeroed_baisies
    
# initialize_zeros(10)
# write test cases for this function <--- use asserts, check that zeored_weights and zeored_baisies are of the right
# type. 

In [59]:
def propagate(X,Y,w,b):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    # Y = (1, examples)
    print("propagate shapes")
    print("X",np.shape(X))
    print("Y",np.shape(Y))
    print("w",np.shape(w))
    print("b",np.shape(b))
    
    w_transposed = w.T
    num_of_examples = np.shape(X)[1]
    
    dot_product = np.dot(w_transposed,X) #dot product of X and w ORDER MATTERS
    z = dot_product + b
    print("z",np.shape(z))
    a = sigmoid(z) #in begining should be 0
    loss = crossEntropyLoss(a,Y)
    #loss = (-1/num_of_examples)*np.sum((1-Y)*np.log(1-a) + (Y *np.log(a))) #log likelyhood cost for logistic regression
    print("loss",loss)
    dw = (1/num_of_examples) * np.dot(X,(a-Y).T) #?
    #
    db = (1/num_of_examples) * np.sum(a-Y) #np.sum collapes input
    #
    loss = np.squeeze(loss)
    #
    grads = {"dw":dw,"db":db}
    return grads,loss    


In [47]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(X, Y,w,b)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

propagate shapes
X (2, 3)
Y (1, 3)
w (2, 1)
b ()
z (1, 3)
(1, 3) a
loss 5.80154531939
dw = [[ 0.99845601]
 [ 2.39507239]]
db = 0.00145557813678
cost = 5.80154531939


In [58]:
def optimizer(w, b, X, Y, learning_rate, num_iterations, print_results=True):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    # Y = (1, examples)
    print("optimizer shapes")
    print("X",np.shape(X))
    print("Y",np.shape(Y))
    print("w",np.shape(w))
    print("b",np.shape(b))
    costs = []
#     print("X in optimizer",np.shape(X),"Y in optimizer",np.shape(Y))
    for i in range(num_iterations):
        grads, cost = propagate(X,Y,w,b)
        print(cost)
        dw = grads["dw"]
        db = grads["db"]
        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)
        if (i % 50 == 0) and (print_results == True):
            costs += [cost]
            print("cost #",i,":", costs)
    if print_results == True:
        print("Finsihed :)" , costs )
    grads = {"dw":dw,"db":db}
    params = {"w":w,"b":b}
    return grads,params,costs

In [49]:
print("shapes before optimizer")
print("X Shape", np.shape(standardized_train_examples),"Y Shape", np.shape(labels_train))
print("Weights Shape",np.shape(w),"Baisies shape",np.shape(b))
optimizer(w,b,standardized_train_examples,labels_train,.1,10)

shapes before optimizer
X Shape (1267500, 320) Y Shape (320,)
Weights Shape (2, 1) Baisies shape ()
propagate shapes
X (1267500, 320)
Y (320,)
w (2, 1)
b ()


ValueError: shapes (1,2) and (1267500,320) not aligned: 2 (dim 1) != 1267500 (dim 0)

In [61]:
def predictor(X,w,b):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    print("predictor shapes")
    print("X",np.shape(X))
    print("w",np.shape(w))
    print("b",np.shape(b))
    w_transposed = w.T
#     print(np.shape(w_transposed),np.shape(X))
#     new_w = w_transposed.reshape(X.shape[0], 1)
#     print(new_w.shape)
#     print(np.reshape(w_transposed, X[0]))
    prediction = (np.dot(w_transposed,X)) + b
    prediction = sigmoid(prediction)
    prediction *= 2.0
    prediction = np.floor(prediction)
#     print(np.shape(prediction))
    return prediction
    


In [ ]:
predict_examples = (standardized_train_examples)
# for item in predict_examples:
#     print(np.shape(item))
predictor(predict_examples,w,b)

In [51]:
def logisticRegression(train_X,train_Y,test_X,test_Y,learning_rate,epoch):
    # train_x = (image_width * image_width * 3, training_examples)
    # train_y = (1, train_examples)
    # test_x = (____, test_examples)
    # test_y = (1, test_examples)

    print(train_X.shape)
    
    num_of_examples =  np.shape(train_Y)[0]
#     train_Y = train_Y.reshape(num_of_examples,1)
#     train_Y = train_Y.T 
    w,b = initialize_zeros(side_length*side_length*3)
#     print("shapes before optimizer",np.shape(train_X),np.shape(train_Y))
    print("shapes before optimizer IIIIIII")
    print("X Shape", np.shape(train_X),"Y Shape", np.shape(train_Y))
    print("Weights Shape",np.shape(w),"Baisies shape",np.shape(b))
    _, params,_ = optimizer(w, b, train_X, train_Y, learning_rate, epoch)
    w,b = params["w"], params["b"]
    prediction_train = predictor(train_X,w,b)
    accuracy_train = np.mean(np.abs(prediction_train - train_Y))
    prediction_test = predictor(test_X,w,b)
    accuracy_test = np.mean(np.abs(prediction_test - test_Y))
    print("Train Accuracy:", accuracy_train)
    print("Actual Accuracy:", accuracy_test)
        

In [62]:
train_X = standardized_train_examples
train_Y = labels_train.T
test_X = standardized_test_examples
test_Y = labels_test.T
learning_rate = .1
epoch = 3
print(train_X.shape,"train_X")
print(train_Y.shape,"train_Y")
print(test_X.shape,"test_X")
print(test_Y.shape,"test_Y")
logisticRegression(train_X,train_Y,test_X,test_Y,learning_rate,epoch)

(1267500, 320) train_X
(320,) train_Y
(1267500, 80) test_X
(80,) test_Y
(1267500, 320)
shapes before optimizer IIIIIII
X Shape (1267500, 320) Y Shape (320,)
Weights Shape (1267500, 1) Baisies shape ()
optimizer shapes
X (1267500, 320)
Y (320,)
w (1267500, 1)
b ()
propagate shapes
X (1267500, 320)
Y (320,)
w (1267500, 1)
b ()
z (1, 320)
(320,)


IndexError: tuple index out of range

Model flow:
(1) logistic Regression
    (2) optimizer
        (3) proprogate 
    (4) predict